# Selenium Driver 를 이용해서 다운로드 
SSS Instagram Web Service  
https://sssinstagram.com/  

# Setup Environment
References:  


# 패키지/모듈 로딩

In [1]:
import re 
from time import sleep
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=2)
import os 

from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import requests 
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [2]:
driver = webdriver.Chrome()
driver 

<selenium.webdriver.chrome.webdriver.WebDriver (session="40f74d267638d4092efda341d7a9c7b3")>

In [9]:
driver.get("https://sssinstagram.com/")

### 컨텐츠타입 입력

In [11]:
def select_contents_type(type='Video'):
    try:
        elem = WebDriverWait(driver, 10).until(
            # EC.presence_of_element_located((By.CLASS_NAME, "pageTunerWrapper__items"))
            EC.presence_of_element_located((By.XPATH, "//a[@href='https://sssinstagram.com/video-downloader']"))
        )
    except Exception as e:
        print("ERROR:", e)
    else:
        print('presence_of_element:', elem)
        elem.click()
        
        # a = elem.find_element(By.LINK_TEXT, type.capitalize())
        # print('a:', a)

        # app = driver.find_element(By.ID, 'app')
        # nav = app.find_element(By.CLASS_NAME, "pageTunerWrapper")
        # print('nav:', nav)

        # type = type.capitalize()
        # print('type:', type)
        # a = nav.find_element(By.LINK_TEXT, type)
        # print('a:', a)
        # a.click()
    

select_contents_type('video')

presence_of_element: <selenium.webdriver.remote.webelement.WebElement (session="40f74d267638d4092efda341d7a9c7b3", element="f.584DF58540FB1D8CAFFF213F88C83AAD.d.A7C4848441A558433529846068F430BA.e.7771")>


### 링크주소 입력

In [19]:
def input_link(url):
    input_box = driver.find_element(By.ID, 'input')
    # print(input_box) 

    input_box.send_keys(url)
    input_box.send_keys(Keys.RETURN)

# input_link("https://www.instagram.com/p/C-hts7migZz/?utm_source=ig_web_copy_link") 
input_link("https://www.instagram.com/reel/C_Ln60OiEmy/?utm_source=ig_web_copy_link")

### META DATA PARSER

In [17]:
def get_body_text():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    app = soup.find(id='app')
    result = app.find('div', class_="search-result")
    text = result.find('p', class_="output-list__caption")
    print(text.get_text())


get_body_text()

Buenas vibras con este palo de canción  sólo hay que ver nuestras caras🫶  @pintopicasso dímelo!   @dismo_media   @lasvegassupercongress #lasvegas   #bachata #luisyandrea #usa #bachatadance #america


### 다운로드

In [18]:
def get_download_urls():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    app = soup.find(id='app')
    result = app.find(class_="search-result")
    _list = result.find('ul', class_="output-list__list")
    # print(_list.prettify())
    urls = []
    for li in _list.find_all('li'):
        d = li.find('a').attrs
        urls.append(d['href'])
    return urls 


def download_file(url, local_filename):
    # Send a GET request to the URL
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Check for HTTP request errors
        # Open a local file with write-binary mode
        with open(local_filename, 'wb') as file:
            # Write the content of the response in chunks
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    print(f"File downloaded: {local_filename}")


from pathlib import WindowsPath

def download(_dir='C:\\Users\\innovata\\Downloads', delay=3):
    _dir = str(WindowsPath(_dir))

    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "search-result"))
        )
    except Exception as e:
        print(f"ERROR | {e}")
    else:
        print('results', results)

        # ul = results.find_element(By.CLASS_NAME, 'output-list__list')
        # print(ul)
        try:
            ul = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "output-list__list"))
            )
        except Exception as e:
            print(f"ERROR | {e}")
        else:
            print('ul', ul)

            urls = get_download_urls()
            filepaths = []
            for i, url in enumerate(urls, start=1):
                print([i, url])

                filepath = os.path.join(_dir, str(i).zfill(2)+'.mp4')
                filepaths.append(filepath)
                download_file(url, filepath)

            print('다운로드 완료')
            
            # 동영상 파일 병합
            clips = []
            for filepath in filepaths:
                clips.append(VideoFileClip(filepath))
            final_clip = concatenate_videoclips(clips)
            out_file = os.path.join(_dir, 'output.mp4')
            final_clip.write_videofile(out_file)
            print('동영상 파일 병합 완료.')

            # 다운로드한 분할 영상은 삭제 
            for filepath in filepaths:
                os.remove(filepath)
            print('분할파일 삭제 완료.')

            print('DONE.')


# urls = get_download_urls()
download()

results <selenium.webdriver.remote.webelement.WebElement (session="40f74d267638d4092efda341d7a9c7b3", element="f.584DF58540FB1D8CAFFF213F88C83AAD.d.6DE6AC4A2DE1A0BBC7D7E0556B2FECCC.e.15134")>
ul <selenium.webdriver.remote.webelement.WebElement (session="40f74d267638d4092efda341d7a9c7b3", element="f.584DF58540FB1D8CAFFF213F88C83AAD.d.6DE6AC4A2DE1A0BBC7D7E0556B2FECCC.e.15445")>
[1, 'https://media.sssinstagram.com/get?__sig=H-dHxLKgQZQ0zt3JnVcMFw&__expires=1725168302&uri=https%3A%2F%2Finstagram.fiev6-1.fna.fbcdn.net%2Fo1%2Fv%2Ft16%2Ff2%2Fm69%2FAn-6Nw-jTM06hDHoYyytonywelGKzQPkyX7J_lQ24uQsxw4RLFBFNxo8urowPI5QJW0cPKJzhdgqREbQzZ1nBK0v.mp4%3Fstp%3Ddst-mp4%26efg%3DeyJxZV9ncm91cHMiOiJbXCJpZ193ZWJfZGVsaXZlcnlfdnRzX290ZlwiXSIsInZlbmNvZGVfdGFnIjoidnRzX3ZvZF91cmxnZW4uY2xpcHMuYzIuMTA4MC5iYXNlbGluZSJ9%26_nc_cat%3D103%26vs%3D1916387882163993_1934979421%26_nc_vs%3DHBksFQIYOnBhc3N0aHJvdWdoX2V2ZXJzdG9yZS9HSUlZNkFJX3ZDMGVwNm9CQUc0UDdxVkVuQThTYnBSMUFBQUYVAALIAQAVAhg6cGFzc3Rocm91Z2hfZXZlcnN0b3JlL0dNWDlRaHZuX

MoviePy - Done.
Moviepy - Writing video C:\Users\innovata\Downloads\output.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\innovata\Downloads\output.mp4
동영상 파일 병합 완료.
분할파일 삭제 완료.
DONE.


In [105]:
from urllib.parse import urlparse
o = urlparse(urls[0])

In [109]:
o.path 

'/get'

In [21]:
def download_insta(url, type='video'):
    driver.get("https://sssinstagram.com/")
    select_contents_type(type)
    input_link(url) 
    download(delay=3)

download_insta("https://www.instagram.com/p/C_TL2pMCS7-/?utm_source=ig_web_copy_link", type='video')

presence_of_element: <selenium.webdriver.remote.webelement.WebElement (session="40f74d267638d4092efda341d7a9c7b3", element="f.584DF58540FB1D8CAFFF213F88C83AAD.d.809B4AB5CE417023AB9B96CA045E2E68.e.20749")>
results <selenium.webdriver.remote.webelement.WebElement (session="40f74d267638d4092efda341d7a9c7b3", element="f.584DF58540FB1D8CAFFF213F88C83AAD.d.8AD11DDD5B3AD5CF88E6F88FA15E9641.e.21904")>
ul <selenium.webdriver.remote.webelement.WebElement (session="40f74d267638d4092efda341d7a9c7b3", element="f.584DF58540FB1D8CAFFF213F88C83AAD.d.8AD11DDD5B3AD5CF88E6F88FA15E9641.e.22121")>
[1, 'https://media.sssinstagram.com/get?__sig=0ozzuWyr7HquTVwkrcxmCA&__expires=1725180088&uri=https%3A%2F%2Finstagram.faep4-3.fna.fbcdn.net%2Fo1%2Fv%2Ft16%2Ff2%2Fm69%2FAn-a4WoetnGIn7cHkII07f9xW5nMSa6acfXo7qFEzC9Q9-Qt4iSj2enVPpxF2MJtK6XaQKVBm_sJpq18tqoC5n3-.mp4%3Fstp%3Ddst-mp4%26efg%3DeyJxZV9ncm91cHMiOiJbXCJpZ193ZWJfZGVsaXZlcnlfdnRzX290ZlwiXSIsInZlbmNvZGVfdGFnIjoidnRzX3ZvZF91cmxnZW4uY2Fyb3VzZWxfaXRlbS5jMi4xMDgwLmJ

MoviePy - Done.
Moviepy - Writing video C:\Users\innovata\Downloads\output.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\innovata\Downloads\output.mp4
동영상 파일 병합 완료.
분할파일 삭제 완료.
DONE.
